# Sheet properties

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import array, logspace
from imblearn import combine as combine
from imblearn import over_sampling as overs
from imblearn import under_sampling as unders
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from toolbox.classification.common import Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
from toolbox.transforms.at_most import LDAAtRatio

# Parameters

In [ ]:
# Advanced parameters
validation = 4
settings = Settings.get_default_dermatology()

# Inputs

In [ ]:
prediction_file = f'Prediction_DataBalancement.pickle'

In [ ]:
inputs = Dermatology.images(modality='Microscopy', data_type='Full')
inputs_spatial = IO.load(f'Extraction_Spatial.pickle')
inputs_freq = IO.load(f'Extraction_Frequency.pickle')
inputs_transfer = IO.load(f'Extraction_Transfer.pickle')

In [ ]:
inputs['Spatial'] = inputs_spatial['Wiltgen']
inputs['Daubechies'] = inputs_freq['DWTDB4Sc1']
inputs['VGG16Avg'] = inputs_transfer['VGG16Avg']
inputs['ResNetAvg'] = inputs_transfer['ResNetAvg']

In [ ]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

# Models

In [ ]:
mains = [('Spatial', [('scale', MinMaxScaler())]),
        ('Daubechies', [('scale', StandardScaler())]),
        ('ResNetAvg', [('scale', MinMaxScaler())]),
        ('VGG16Avg', [('scale', MinMaxScaler()), ('reduction', LDAAtRatio(n_components=0.95))])]

In [ ]:
clf = ('clf', SVC(kernel='linear', probability=True))
clf_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [ ]:
strategies = []
strategies.append(('NoStrategy', ('strategy', None)))
# Random Over Sample
strategies.append(('RandomOS',('strategy', overs.RandomOverSampler())))
# Random Under Sample
strategies.append(('RandomUS',('strategy', unders.RandomUnderSampler())))
strategies.append(('SMOTEENN',('strategy', combine.SMOTEENN())))
strategies.append(('SMOTETomek',('strategy', combine.SMOTETomek())))

In [ ]:
processes = []
for datum, main in mains:    
    for strategy_name,strategy in strategies:
        process = main.copy()
        if not strategy_name == 'NoStrategy':
            process.extend([strategy, clf])
        else:
            process.extend([clf])
        processes.append((datum, f'{datum}{strategy_name}', Pipeline(process)))

# Evaluation

In [ ]:
for datum, name, process in processes: 

    # Start evaluation       
    print(f'{name} performed...', end='\r')
    Tools.evaluate(inputs, {'datum': datum, 'label_encode': 'LabelEncode'}, process, name, distribution=clf_params)
    print(f'{name} achieved !', end='\r')  

    # Save
    IO.save(inputs, prediction_file)

# Scores and ROC

In [ ]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(prediction_file)

# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))
    
for datum, name, process in processes:  
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LabelEncode', 'eval': name}, label_encoder)],
                                                                        title=[f'Test - {name}'])))